# <div style="text-align:center;">**Capuchin Bird Signal Processing**</div>

# 1. Load Dependencies

Since I am using a venv, I am installing the required packages using pip

In [8]:
%pip install tensorflow==2.8.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.8.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==2.8.0


In [2]:
%pip install matplotlib
%pip install tensorflow==2.12.0
%pip install tensorflow_io==0.29.0

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/272.9 MB ? eta -:--:--
     --------------------------------------- 0.6/272.9 MB 36.9 MB/s eta 0:00:08
     --------------------------------------- 2.0/272.9 MB 25.1 MB/s eta 0:00:11
     --------------------------------------- 3.2/272.9 MB 25.6 MB/s eta 0:00:11
      -------------------------------------- 4.3/272.9 MB 25.2 MB/s eta 0:00:11
      -------------------------------------- 5.2/272.9 MB 25.8 MB/s eta 0:00:11
      -------------------------------------- 5.7/272.9 MB 21.5 MB/s eta 0:00:13
      -------------------------------------- 6.3/272.9 MB 22.3 MB/s eta 0:00:12
      -------------------------------------- 6.3/272.9 MB 22.3 MB/s eta 0:00:12
      -------------------------------------- 6.3/272.9 MB 22.3 MB/s eta 0:00:12
      -------------------------------------- 6.3/272.9 MB 22.3 MB/s eta 0:00:12
      -------------------------------------- 6.3/272.9 MB 22.

In [3]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

In [4]:
print('tensorflow version:', tf.__version__)
print('tensorflow_io version:', tfio.__version__)

tensorflow version: 2.12.0
tensorflow_io version: 0.29.0


In [5]:
%pip list

Package                      Version
---------------------------- ----------
absl-py                      2.0.0
asttokens                    2.4.1
astunparse                   1.6.3
cachetools                   5.3.2
certifi                      2023.11.17
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.0
contourpy                    1.2.0
cycler                       0.12.1
debugpy                      1.8.0
decorator                    5.1.1
executing                    2.0.1
flatbuffers                  23.5.26
fonttools                    4.46.0
gast                         0.4.0
google-auth                  2.25.2
google-auth-oauthlib         1.0.0
google-pasta                 0.2.0
grpcio                       1.60.0
h5py                         3.10.0
idna                         3.6
ipykernel                    6.27.1
ipython                      8.18.1
jax                          0.4.23
jedi                         0.19.1

# 2. Data Collection

In [6]:
audio = tfio.audio.AudioIOTensor('data/Parsed_Not_Capuchinbird_Clips/Crickets-chirping-0.wav')

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['c:\\Users\\PanSt\\OneDrive\\Υπολογιστής\\Coding\\Github Projects\\Capuchin-Bird-Audio-Count\\.venv\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so']
caused by: ['c:\\Users\\PanSt\\OneDrive\\Υπολογιστής\\Coding\\Github Projects\\Capuchin-Bird-Audio-Count\\.venv\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so not found']

In [2]:
def load_data(file):
    # read the .wav file
    wav, sample_rate = tf.audio.decode_wav(tf.io.read_file(file), desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

## 3. Visualization

In [3]:
capu = load_data(os.path.join('data', 'Parsed_Capuchinbird_Clips', 'XC114131-0.wav'))
non_capu = load_data('data/Parsed_Not_Capuchinbird_Clips/Crickets-chirping-0.wav')
forest_mix = load_data('data/Forest Recordings/recording_00.wav')

NotImplementedError: in user code:

    File "c:\Users\PanSt\OneDrive\Υπολογιστής\Coding\Github Projects\Capuchin-Bird-Audio-Count\.venv\Lib\site-packages\tensorflow_io\python\ops\audio_ops.py", line 458, in f
        return core_ops.io_audio_resample(
    File "c:\Users\PanSt\OneDrive\Υπολογιστής\Coding\Github Projects\Capuchin-Bird-Audio-Count\.venv\Lib\site-packages\tensorflow_io\python\ops\__init__.py", line 88, in __getattr__
        return getattr(self._load(), attrb)
    File "c:\Users\PanSt\OneDrive\Υπολογιστής\Coding\Github Projects\Capuchin-Bird-Audio-Count\.venv\Lib\site-packages\tensorflow_io\python\ops\__init__.py", line 84, in _load
        self._mod = _load_library(self._library)
    File "c:\Users\PanSt\OneDrive\Υπολογιστής\Coding\Github Projects\Capuchin-Bird-Audio-Count\.venv\Lib\site-packages\tensorflow_io\python\ops\__init__.py", line 69, in _load_library
        raise NotImplementedError(

    NotImplementedError: unable to open file: libtensorflow_io.so, from path